# Network Load Balancer

## Elastic Load Balancer의 종류
- Application Load Balancer
    - 똑똑
    - OSI Model Layer 7
    - 트래픽을 모니터링하여 라우팅 가능
        - 예 : image.sample.com -> 이미지 서버로, web.sample.com -> 웹 서버로 트래픽 분산
- Network Load Balancer
    - 빠름
    - OSI Model Layer 4
    - TCP, UDP 기반 빠른 트래픽 분산
    - Elastic IP 할당 가능
- Classic Load Balancer
    - 옛날
    - 예전에 사용되던 타입, 현재는 잘 사용하지 않음
- Gateway Load Balancer
    - 먼저 트래픽을 체크
    - OSI Model Layer 3
    - 가상 어플라이언스 배포/확장 관리를 위한 서비스

### Application Load Balancer
- OSI Model Layer 7
    - header, cookie, domain 이해 가능
        - header를 변경하거나 domain 기반 라우팅
    - health check 역시 Layer 7 프로토콜 기반(http)
- HTTP / HTTPs 프로토콜만 지원
    - 지속적인 연결 없이 요청 시에만 연결 수립
- 다양한 규칙 기반으로 여러 대상그룹에 분배 가능
- IP 고정 불가능
    - IPAM을 활용할 수 있음(고급 기능)
    - Network Load Balancer 뒤에 Application Load Balancer를 놓는 것을 추천

### Network Load Balancer
- OSI Model Layer 4
    - TCP, UDP, TLS 등
    - HTTP 프로토콜 이해 불가능
        - 즉 Header, Cookie, 세션 등 활용 불가능
- ALB에 비한 장점
    - 매우 빠른 속도 : 초당 수백만건 이상 요청 처리
    - 연결 지속 가능
    - Elastic IP 활용 가능 : IP 고정 가능
- 주요 사용 사례:
    - 빠른 TCP / UDP 연결이 필요한 어플리케이션 : 게임, SSH, 실시간성이 필요한 어플리케이션, 금융 등

### Network Load Balancer의 고정 IP
- 본래 ELB의 경우 DNS기반으로 접속하며 프로비전 된 Node의 IP 목록 중 하나를 전달
- NLB의 경우 이 Node중 하나를 Elastic IP로 고정 가능
- 이미 생성된 NLB에 EIP를 붙이고 싶은 경우 노드 재 생성 필요
    - 연동 서브넷을 제거하고 다시 생성 필요

### 데모 - NLB 프로비전 및 테스트
- EC2 + nodejs 기반 TCP 서버를 클라우드포메이션으로 프로비전
    - 간단하게 TCP로 접속한 클라이언트에게 1초마다 현재 시각을 전달하는 서버
    - 3000포트를 활용하며 해당 포트를 여는 보안그룹 역시 프로비전
- 대상 그룹 추가 및 프로비전한 EC2 추가
- Nodejs 기반 클라이언트로 접속 확인
- 이후 고정 IP를 추가하여 고정 아이피로 접속 확인

### 데모 순서
1. 클라우드 포메이션으로 인스턴스 프로비전
2. NLB 생성
    - 대상 그룹부터 생성 : TCP 프로토콜, 3000 포트
    - CF로 프로비전한 인스턴스를 대상 그룹에 추가
    - NLB 생성 : 리스너 포트 3000
3. NLB 테스트
    - client.js 내용 수정 : NLB의 DNS 이름 사용
4. Elastic IP 붙이기
    - 탄력적 IP 생성
    - NLB의 네트워크 매핑에서 서브넷을 하나 지웠다가 새로 만들어보자 : EIP 적용 가능
5. NLB 테스트
    - 호스트 주소에 EIP 입력
    - 연결 불가 : NLB의 노드가 다른 가용영역에 접근할 수 없기 떄문
        - EIP는 가용영역 d의 노드에 붙었는데, 인스턴스는 가용영역 c에 있음 : 교차 영역 로드 밸런싱 활성화가 필요
6. 리소스 정리 : 로드밸런서, CF스택